# Bike Share Raw Data

In London, UK and Taipei, Taiwan, there are bike-share systems using docks. A rental event is recorded at least as a start station, an end station, and the two corresponding times. This defines a spatiotemporal data set, or in other words, a multi-dimensional time-series on a graph.

The raw data files have been retrieved from the respective bike-sharing system operators, parsed and moderately reformatted (e.g. making column names uniform over all data), then saved into one CSV file per city, rows sorted on start date. Only raw data from 2017 and up until mid-2020 are included.

The raw data sources are:
* https://cycling.data.tfl.gov.uk
* https://data.taipei/#/dataset?topic=topic-transportation

For examples of analyses of this data as blog posts, follow these links:

* Build usage baseline and contrast how COVID-19 impact usage when and where: https://towardsdatascience.com/responses-to-covid-19-in-taipei-and-london-as-revealed-by-bike-sharing-5fcb215f2341
* Simple graph convolutional neural networks for forecasting rental events: https://towardsdatascience.com/london-bike-ride-forecasting-with-graph-convolutional-networks-aee044e48131
* Descriptive analysis and visualization for London data: https://medium.com/@AJOhrn/data-footprint-of-bike-sharing-in-london-be9e11425248

Academic article using the same type of data:
https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9093851

Next I provide a handful of helpful commands to enable analysis.

## Load Data and Basic Cleaning and Extension

In [ ]:
import pandas as pd
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Start with London, and load all bike-share data for the Santander London system, January 2017 up until mid-2020. Parse the two time columns as datetime types.

In [ ]:
df_london = pd.read_csv('/kaggle/input/bikeshare-usage-in-london-and-taipei-network/london.csv', parse_dates=[3,6], infer_datetime_format=True)
df_london.shape

The data includes rare rental events that are started but not ended. For some analysis these are best removed.

In [ ]:
df_london = df_london.loc[~df_london.isnull().any(axis=1)]
df_london = df_london.astype({'bike_id' : 'int32', 'end_station_id' : 'int32', 'start_station_id' : 'int32'})
df_london.shape

Augment datetime data to make it easier for binning and applying conditions with respect to time.

In [ ]:
df_london.loc[:, 'year'] = df_london['start_rental_date_time'].dt.year
df_london.loc[:, 'month'] = df_london['start_rental_date_time'].dt.month
df_london.loc[:, 'week'] = df_london['start_rental_date_time'].dt.isocalendar().week
df_london.loc[:, 'day'] = df_london['start_rental_date_time'].dt.day
df_london.loc[:, 'hour'] = df_london['start_rental_date_time'].dt.hour
df_london.loc[:, 'dayofweek'] = df_london['start_rental_date_time'].dt.dayofweek
df_london.loc[:, 'satsun'] = df_london['dayofweek'].map({0: False, 1: False, 2: False, 3: False, 4: False, 5: True, 6:True})
df_london.shape

A handful of stations are very rarely part of any rental events. In some cases these are temporary stations or stations that become decommissioned. In some analysis these low count stations are best removed.

In [ ]:
below_or_above_stations = df_london['start_station_name'].value_counts() < 500
low_stations = below_or_above_stations.loc[below_or_above_stations].index
df_london = df_london.loc[~df_london['start_station_name'].isin(low_stations)]
df_london = df_london.loc[~df_london['end_station_name'].isin(low_stations)]
df_london.shape

## Analysis Can Begin (London)

On this dataset descriptive analysis can be made. Examples of that include seasonal effects, weekday verus weekend, particular stations that change the most between years etc. A simple example is the number of rental events in the months of a particular year, say 2018.

In [ ]:
import seaborn as sns
per_month_2018_group = df_london.loc[df_london['year'] == 2018].groupby('month')
month_counts = per_month_2018_group.count()

sns.barplot(x=month_counts.index, y=month_counts.iloc[:,0])

Another example is to analyze time-series for specific stations within a day, like station 14 (a major commuter station in London) during weekdays.

In [ ]:
weekday_14_start = df_london.loc[(df_london['start_station_id'] == 14) & (df_london['satsun'] == False)]
weekday_14_end = df_london.loc[(df_london['end_station_id'] == 14) & (df_london['satsun'] == False)]
weekday_14_start = weekday_14_start.groupby('hour').count()
weekday_14_end = weekday_14_end.groupby('hour').count()
weekday_14_data = pd.DataFrame({'start_counts' : weekday_14_start.iloc[:,0],
                                'end_counts' : weekday_14_end.iloc[:,0]})

sns.lineplot(data=weekday_14_data)

Longitude and latitude data is also available for each station.

In [ ]:
df_london_geo = pd.read_csv('/kaggle/input/bikeshare-usage-in-london-and-taipei-network/london_stations.csv')
df_london_geo.head(5)

Additional analysis and data mining are easy to image, some of which can be found in this blog post:  https://medium.com/@AJOhrn/data-footprint-of-bike-sharing-in-london-be9e11425248

More advanced analysis can use the data to model the distributions of expected rental events at different stations and/or time of day. For example, it has been found that weather matters a great deal to rental events, at least extreme weather. A Bayesian analysis that models what weather conditions implies for the distribution of rental events, can be attempted by cross-referencing current data with weather data for London.

**Note that the data contains a major concept shift in March 2020** when the COVID-19 pandemic impacts London. That itself can be subject of analysis, as shown in this blog post:  https://towardsdatascience.com/responses-to-covid-19-in-taipei-and-london-as-revealed-by-bike-sharing-5fcb215f2341

Have fun!

## Graph Analysis

The data can be subject to graph analysis, with each station a vertex in the graph and the edge weight between two vertices related to the number of rental events involves the corresponding pair of stations. The directed weighted adjacency matrix is the key quantity.

In [ ]:
station_pair_group = df_london.groupby(['start_station_name', 'end_station_name'])
station_pair_count = station_pair_group.count().iloc[:,0]

There are multiple valid relations between edge weight and number of rental events involving the corresponding pair of stations. One is to relate the weight to the percentage of all rental events from the given start station that flows through the given edge.

In [ ]:
total_by_start_station = station_pair_count.groupby('start_station_name').sum()
rel_weight = 100.0 * station_pair_count.div(total_by_start_station, level=0)

Rare connections can sometimes be removed in order to make the graph representation sparser.

In [ ]:
rel_weight = rel_weight.loc[rel_weight > 1.0]

In some analysis (and especially visualizations) the adjacency matrix needs to account for no adjacency. The `rel_weight` data frame accounts for non-zero adjacencies only. The full matrix is thus constructed by adding zeros.

In [ ]:
import numpy as np
stations = np.union1d(df_london['start_station_name'].unique(), df_london['end_station_name'].unique())
mindex_square = pd.MultiIndex.from_product([stations, stations])
rel_weight_square = rel_weight.reindex(index=mindex_square, fill_value=0.0)
rel_weight_square = rel_weight_square.unstack()
sns.heatmap(rel_weight_square.values, vmin=0.0, vmax=5.0, cmap="YlGnBu")

Some network analysis can be done by representing the directed weighted graph as a `DiGraph` instance from the `networkx` library.

In [ ]:
import networkx as nx
dg = nx.DiGraph()
dg.add_nodes_from(stations)

The edge definitions in `networkx` require edges to be defined as a three-membered tuple.

In [ ]:
edge_weights_dict = rel_weight.to_dict()
edge_weights_data = [(key1, key2, val) for (key1, key2), val in edge_weights_dict.items()]
dg.add_weighted_edges_from(edge_weights_data)

A range of analysis can be run using methods in `networkx.algorithms`. One example is the node clusterings.

In [ ]:
node_clusterings = nx.algorithms.cluster.clustering(dg)
sorted(node_clusterings.items(), key=lambda kv: kv[1])

## Graph Convolution Networks

A continuation of the graph analysis is to employ Graph Convolution Networks (GCNs) to model the rental time-series. One possible application is in forecasting, taking into account the known spatial relations between the stations, as illustrated in this blog post: https://towardsdatascience.com/london-bike-ride-forecasting-with-graph-convolutional-networks-aee044e48131 Other applications can be classification of stations on basis of its "when-and-where" place in the bike-sharing system, or maybe system-wide properties, like weather, special events, highly resolved consumer spending in SoHo... Possible options are in no shortage, useful options is another question.

One possibility is to use the [PyTorch Geometric library](https://pytorch-geometric.readthedocs.io/en/latest/index.html). The graph of the bike system has then to be represented as required by PyTorch Geometric. 

Unless installed already, run the `pip install` and if GPU is used, replace `cpu` with the CUDA version, like `cu101`. See the PyTorch Geometric installation help page for details.

In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cpu.html
!pip install torch-geometric

In [ ]:
import torch
from torch_geometric.data import Data

As illustration I create dummy node input features, one per node, which should be replaced with *input features* to the neural network. These could, for example, represent the number of rental events at said station in the previous hour. 

I also create dummy one-feature-per-node output. The dimension of the output should reflect the prediction task, so a single system-wide scalar feature would instead correspond to a tensor containing just one value.

In [ ]:
stations_dummy_inp_features = torch.tensor([np.random.ranf() for s in stations])
stations_dummy_out_features = torch.tensor([np.random.ranf() for s in stations])

The adjacency matrix is represented in the COO format, and the weights are made into edge attributes. For those GCN methods that use weights, that is what PyTorch Geometric expects.

In [ ]:
edge_s = []
edge_t = []
edge_attr = []
for row, value in rel_weight.items():
    edge_s.append(list(stations).index(row[0]))
    edge_t.append(list(stations).index(row[1]))
    edge_attr.append(value)
    
edge_index = torch.tensor([edge_s, edge_t])
edge_attr = torch.tensor(edge_attr).unsqueeze(1)

Put it all together into a single instance of input features on a graph and corresponding output features.

In [ ]:
pyg_graphtensor = Data(x=stations_dummy_inp_features, y=stations_dummy_out_features,
                       edge_index=edge_index, edge_attr=edge_attr)

The neural network model has to be built after this, along with a dataset comprised of several instances of `Data`. These are details for another notebook.

Because the bike-share data imply a natural graph structure, the inductive bias of GCN should be able to represent some spatial features of the bike-sharing data, in case these are relevant to the prediction objective that is. At least that is a hypothesis worth exploring.

## Analysis Continues (Taipei)

The data for Taipei bike-sharing system can undergo the same analysis as outlined above for London. The data for Taipei is slightly different, however. I will highlight these differences only.

Note that this is rather large CSV file, so loading in chunks or rows subsets may be needed.

In [ ]:
df_taipei = pd.read_csv('/kaggle/input/bikeshare-usage-in-london-and-taipei-network/taipei.csv', parse_dates=[0,2,5], infer_datetime_format=False, 
                        nrows=10000)
df_taipei.shape

In [ ]:
df_taipei.loc[:, 'year'] = df_taipei['start_rental_date_hour'].dt.year
df_taipei.loc[:, 'month'] = df_taipei['start_rental_date_hour'].dt.month
df_taipei.loc[:, 'week'] = df_taipei['start_rental_date_hour'].dt.isocalendar().week
df_taipei.loc[:, 'day'] = df_taipei['start_rental_date_hour'].dt.day
df_taipei.loc[:, 'hour'] = df_taipei['start_rental_date_hour'].dt.hour
df_taipei.loc[:, 'dayofweek'] = df_taipei['start_rental_date_hour'].dt.dayofweek
df_taipei.loc[:, 'satsun'] = df_taipei['dayofweek'].map({0: False, 1: False, 2: False, 3: False, 4: False, 5: True, 6:True})
df_taipei.shape

In [ ]:
df_taipei.head(8)

Contrasts with London dataset:
* Taipei bike-sharing is larger than London (a bit over double the number of rental events).  
* The stations are not assigned a numeric ID by the operator. As part of the analysis that can be added.
* The station names are in traditional chinese. In order to enable easy filtering for users who cannot easily read or write such characters, the `pinyin` library has been used to create basic Pinyin Romanizations of the station names. This is *not* the same as a translation, and the strings are rather clunky.
* The times for start and end of rental is only resolved up to the hour (hence the suffix `_hour` rather than `time`). Note that duration of the bike rental is in units of seconds, however.

Otherwise the data tracks the same transport activity as the data set for London. Comparison between the two cities can therefore be done along across many different features. For example, in a previous comparison it has been found that the rental events across stations in the two systems distribute quite distinctly over time of day, see the discussion about MDS and Jensen-Shannon in this blog post:  https://towardsdatascience.com/responses-to-covid-19-in-taipei-and-london-as-revealed-by-bike-sharing-5fcb215f2341

Good luck with the exploration.